In [17]:
# #사용함수
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import datetime
import json
from selenium.webdriver.common.keys import Keys
import re
import random
import os

class Login:
    def driver_s(self):

        path = "C:\python_temp\chromedriver_win32_95.0\chromedriver.exe"
        self.driver = webdriver.Chrome(path)
        self.driver.get("https://www.instagram.com/accounts/login/")
        time.sleep(2)

        #전체화면
        self.driver.maximize_window()
        time.sleep(1)



    def login_make(self,driver):
        #json으로 cookie를 cookie.json 저장하기
        #한번은 실행하여 json 저장해야함

        username=''#아이디
        password= ''#비밀번호

        #아이디란 선택후 입력
        self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(username)
        #비밀번호란 선택후 입력
        self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(password)
        self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
        time.sleep(2)
        #파업창 닫기
        try:
            self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
        except:
            pass
        time.sleep(2)
        try:
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
        except:
            pass
        time.sleep(2)

        #json으로 현 쿠키값 저장하기
        with open('cookies.json','w') as cookie_json_w:
            json.dump(self.driver.get_cookies(),cookie_json_w)

    def login(self,driver):

        #pickle로 cookie 불러오기
        cookies=json.load(open('cookies.json','r'))
        for cookie in cookies:
            self.driver.add_cookie(cookie)
        time.sleep(1)
        a.refresh() #새로고침

        time.sleep(2)
        try:
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
            time.sleep(2)
        except:
            pass

In [10]:

#검색어 입력

while True:
    search=input('검색할 내용을 입력하세요 : ')
    if search=='':
        print('검색어를 입력해주세요')
        continue
    else:
        while True:    
        #크롤링할 갯수
            try:
                cnt_=int(input('크롤링할 개수를 입력해주세요 : '))
            except ValueError:
                print('정수로 입력해주세요')
                continue
            else:
                break
        break

# #크롬실행
# driver_s()
# time.sleep(1)

# #json이 존재유무에 따라 로그인
# if os.path.exists('cookies.json'):
#     login(driver)
# else:
#     login_make(driver)

# time.sleep(3)

# #입력한 검색어를 인스타에서 검색
# elements=driver.find_element_by_class_name('XTCLo')
# elements.send_keys('#'+search)
# time.sleep(1)
# elements.send_keys(Keys.ARROW_DOWN)
# time.sleep(1)
# elements.send_keys(Keys.ENTER)
# time.sleep(5)

검색할 내용을 입력하세요 : 
검색어를 입력해주세요
검색할 내용을 입력하세요 : 맛집
크롤링할 개수를 입력해주세요 : 
정수로 입력해주세요
크롤링할 개수를 입력해주세요 : 10


In [25]:
#데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'location': [],
              'date': [],
              'like': [],
              'text': [],
              'hashtag': [],
              'img': []}

#이미 게시물을 눌렀놨을 경우 닫기
try: 
    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(0.05)
except:
    pass

#최근 게시물탭에 첫번째게시물을 선택
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div[1]/div[1]/a/div[1]/div[2]').click()
time.sleep(1)


cnt=1
while cnt<cnt_:
    time.sleep(0.05)

    page=driver.page_source
    soup=BeautifulSoup(page,'html.parser')
    time.sleep(0.05)
    
    # 화면을 뜨지 않을 경우 딜레이를 주기위한
    try:
        soup.find('article','M9sTE').get_text()
    except AttributeError:
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/a[1]').click()
        time.sleep(random.randrange(10,15))
    else:
    
        #유저아이디
        try:
            user=soup.find('article','M9sTE').find('div','e1e1d').find('a').get_text()
        except AttributeError:
            user=''
        insta_dict['id'].append(user)
        time.sleep(0.05)

        #지역
        try:
            location=soup.find('article','M9sTE').find('div','JF9hh').find('a').get_text()
        except AttributeError:
            location=''
        insta_dict['location'].append(location)
        time.sleep(0.05)

        #이미지,영상
        #영상일 경우 
        if 'video' in soup.find('article','M9sTE'):
            try:
                img_src=soup.find('article','M9sTE').find('video')
            except AttributeError:
                img=''
            img=img_src['src']
            insta_dict['img'].append(img)
            time.sleep(0.05)
        #영상이 아닐 경우
        else:
            try:
                img_src=soup.find('article','M9sTE').find('img')
            except AttributeError:
                img=''
            img=img_src['src']
            insta_dict['img'].append(img)
            time.sleep(0.05)

        #해시태그
        hasht2=[]
        try:
            hashts=soup.find('article','M9sTE').find_all('a','xil3i')
        except AttributeError:
            hasht=''
        for i in hashts:
            hasht=i.get_text()
            hasht2.append(hasht)
        insta_dict['hashtag'].append(hasht2)
        time.sleep(0.05)

        #게시글
        try:
            posts=soup.find('article','M9sTE').find('div','X7jCj').find('div','C4VMK').find_all('span')[1]
        except AttributeError:
            post=''
        else:
            try:
                for tag in posts.find_all('a'):
                    tag.extract()
                post=posts.get_text()
            except AttributeError:
                post=''
            insta_dict['text'].append(post)
            time.sleep(0.05)

        #좋아요
        likes=soup.find('article','M9sTE')
        try:
            like=likes.find('a','zV_Nj').get_text()
        except AttributeError:
            like=''
        insta_dict['like'].append(like)
        time.sleep(0.05)

        #게시날짜
        day=soup.find('article','M9sTE').find('time','_1o9PC')['datetime']
        insta_dict['date'].append(day)
        time.sleep(0.05)

        print(cnt)
        #현 개수 카운트
        cnt+=1
        
        #다음 버튼 누르기
        driver.find_element_by_css_selector('body > div._2dDPU.QPGbb.CkGkG > div.EfHg9 > div > div > div.l8mY4 > button > div > span > svg').click()
        time.sleep(random.randrange(0,5))
    
#json으로 저장하기
with open(search+'.json','w') as f:
    json.dump(insta_dict,f,indent=4)


1
2
3
4


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[1]/div/div/a[1]"}
  (Session info: chrome=95.0.4638.54)
